In [8]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd

import utils
import models

print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [9]:
MAIN_PATH = "/mnt/homeGPU/fcastro/lulc/"

#data_dir = "../reduced_data/"
data_dir = MAIN_PATH + "data/"
batch_size = 32
img_height = 224
img_width = 224

water_correction=True

submission_dir = MAIN_PATH + "submissions/"
submission_file = "ensemble.csv"
submission_path = submission_dir + submission_file
new_submission_file = "ensemble_waterpcasvm.csv"
new_submission_path = submission_dir + new_submission_file

weights_dir = MAIN_PATH + "weights/"

In [10]:
test_data, test_labels, test_names = utils.load_test_data(data_dir, norm=False)
test_labels = tf.one_hot(test_labels, 29).numpy()
int_test_labels = np.argmax(test_labels, axis=-1)

In [11]:
#model_names = [
#    "mobilenetv3large_v3-1_ft1_DA4_w.h5", "mobilenetv3large_v3-1_ft1_DA4.h5", "mobilenetv3large_v2-1_ft1_DA4.h5", 
#    "mobilenetv3large_v0_ft1_DA0.h5", "mobilenetv3large_v1_ft1_DA4.h5"                
#                ]

model_names = [ "mobilenetv3large_v3-1_ft1_DA4_w.h5" ]

#subm_hist = pd.read_csv("submission_history.csv")
#model_names = (subm_hist[subm_hist['test_acc']>0.95])['submission_name']
#model_names = [f"{m}.h5" for m in model_names]

print(model_names)


['mobilenetv3large_v3-1_ft1_DA4_w.h5']


In [12]:
preds_array = []
for name in model_names:
    print(name)
    base_model_name = name.split("_")[0]
    top_model_name = name.split("_")[1]
    ft_name = name.split("_")[2]
    if ft_name=="ft0":
        ft_mode = 0
    else:
        ft_mode = 1
    load_weights_path = weights_dir + name

    model = models.build_model(base_model_name, top_model_name, ft_mode)
    model.load_weights(load_weights_path)

    prep_fn = models.get_prep_fn(base_model_name)
    prep_test_data = prep_fn(test_data)

    preds = model.predict(prep_test_data, 8)
    preds_array = preds_array + [preds]
    preds_argmax = np.argmax(preds, axis=-1)
    acc = accuracy_score(int_test_labels, preds_argmax)
    print(f"Test Acc.: {acc}")

preds_array = np.array(preds_array)
mean_preds = np.mean(preds_array, axis=0)
mean_preds_argmax = np.argmax(mean_preds, axis=-1)

acc = accuracy_score(int_test_labels, mean_preds_argmax)
print(f"Final test Acc.: {acc}")
true_preds = mean_preds_argmax+1

mobilenetv3large_v3-1_ft1_DA4_w.h5
efficientnet preprocessing
Test Acc.: 0.9585908529048207
Final test Acc.: 0.9585908529048207


In [13]:
np.unique(mean_preds_argmax)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28])

In [14]:
train_data, train_labels = utils.load_train_data(data_dir, norm=False)
idx = np.logical_or(train_labels==20, train_labels==21)
water_train_data = train_data[idx]
water_train_data = water_train_data/255.0
water_train_labels = train_labels[idx]

In [ ]:
pipe = utils.train_water_svm(water_train_data, water_train_labels)

In [15]:
idx_to_modify = np.logical_or(mean_preds_argmax==20, mean_preds_argmax==21)

In [20]:
modify_preds_argmax = mean_preds_argmax.copy()
norm_test_data = test_data/255.0
for i in range(len(mean_preds_argmax)):
    lab = mean_preds_argmax[i]
    if lab==20 or lab==21:
        img = norm_test_data[i]
        x = img.reshape(-1)
        x = np.expand_dims(x, 0)
        p = pipe.predict(x)[0]
        modify_preds_argmax[i] = p
        print(f"old pred: {lab}, new pred: {p}")
true_preds = modify_preds_argmax+1
acc = accuracy_score(int_test_labels, modify_preds_argmax)
print(f"Final test Acc. (after water modification): {acc}")

(150528,)
(1, 150528)
old pred: 20, new pred: 20
(150528,)
(1, 150528)
old pred: 21, new pred: 21
(150528,)
(1, 150528)
old pred: 20, new pred: 20
(150528,)
(1, 150528)
old pred: 20, new pred: 20
(150528,)
(1, 150528)
old pred: 20, new pred: 20
(150528,)
(1, 150528)
old pred: 20, new pred: 20
(150528,)
(1, 150528)
old pred: 20, new pred: 21
(150528,)
(1, 150528)
old pred: 20, new pred: 20
(150528,)
(1, 150528)
old pred: 20, new pred: 20
(150528,)
(1, 150528)
old pred: 20, new pred: 21
(150528,)
(1, 150528)
old pred: 20, new pred: 20
(150528,)
(1, 150528)
old pred: 20, new pred: 20
(150528,)
(1, 150528)
old pred: 20, new pred: 20
(150528,)
(1, 150528)
old pred: 20, new pred: 20
(150528,)
(1, 150528)
old pred: 20, new pred: 21
(150528,)
(1, 150528)
old pred: 20, new pred: 20
(150528,)
(1, 150528)
old pred: 20, new pred: 21
(150528,)
(1, 150528)
old pred: 20, new pred: 20
(150528,)
(1, 150528)
old pred: 21, new pred: 21
(150528,)
(1, 150528)
old pred: 20, new pred: 20
(150528,)
(1, 150528

In [ ]:

print(f"Writing {new_submission_path}")
d = {'id.jpg': test_names, 'label': true_preds}
df = pd.DataFrame(data=d)
df.to_csv(new_submission_path, index=False)
print("Done")